esquema què sha de fer:
1. Importar dades
2. Validar dades: evitar dades repetides, evitar dades sense sentit, localitzar nulls/nan
3. Netejar i corregir dades
4. Generar/modificar columnes noves si cal
5. Normalitzar cada columna
6. Estudiar diferents models de cluster i escollir el millor
7. Estudiar les raons dels clusters graficament (interpretar-los)
8. Generar una metodologia per a tenir un model que puguin fer servir des de back end

# Index   

0. [Importing libraries](##Importing-libraries) 


1. [Exploratory Data Analysis](#Exploratory-Data-Analysis)

2. [Data Enginnering](#Data-Engineering)
 
3. [Model Screening with Pycaret](#Model-screening-with-Pycaret)

4. [Quadratic Discriminant Analysis. Hyperparameter tunning with Gridsearch](##Quadratic-Discriminant-Analysis.-Hyperparameter-tunning-with-Gridsearch)
5. [Prediction of test_X](##test_X-prediction)



### Importing libraries

In [4]:
#Data exploration & manipulation
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from collections import Counter
import seaborn as sns
import matplotlib.style as style 
import datetime as dt
import os
import matplotlib.pyplot as plt
import inspect

#ML Models search and optimisation
from pycaret.classification import * 
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, confusion_matrix
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.decomposition import PCA

# Exploratory Data Analysis

### Data Extraction

In [5]:
products_df = pd.read_csv(r'C:\Users\march\000 Treball Ju`yter 000\Hackathons\Hackathon_Decathlon_ITAcademy\Docs Inicials\prods_IT.csv', index_col=None)
products_df.head(2)
products_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4866 entries, 0 to 4865
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   modeild               4866 non-null   int64  
 1   weblabel              4866 non-null   object 
 2   brand                 4866 non-null   object 
 3   catchline             4866 non-null   object 
 4   productnature         4866 non-null   object 
 5   family                4866 non-null   object 
 6   Review-AverageRating  4866 non-null   float64
 7   DisplayPrice-Amount   4866 non-null   float64
 8   ListPrice-Amount      4866 non-null   float64
dtypes: float64(3), int64(1), object(5)
memory usage: 342.3+ KB


In [6]:
users_df = pd.read_csv(r'C:\Users\march\000 Treball Ju`yter 000\Hackathons\Hackathon_Decathlon_ITAcademy\Docs Inicials\users_IT.csv', index_col=None)
products_df.columns

Index(['modeild', 'weblabel', 'brand', 'catchline', 'productnature', 'family',
       'Review-AverageRating', 'DisplayPrice-Amount', 'ListPrice-Amount'],
      dtype='object')

In [7]:
products_df.columns


Index(['modeild', 'weblabel', 'brand', 'catchline', 'productnature', 'family',
       'Review-AverageRating', 'DisplayPrice-Amount', 'ListPrice-Amount'],
      dtype='object')

In [8]:

recomendations_df = pd.read_csv(r'C:\Users\march\000 Treball Ju`yter 000\Hackathons\Hackathon_Decathlon_ITAcademy\Docs Inicials\recomendations_train_IT.csv', index_col=None)
recomendations_df

,reco_id,time_reco,id_sender,id_reciever,id_product,product_bought
0,5dc916dc-6fa3-488f-8ad2-fd14100fa113,2008-02-09 17:58:24,5565ac1e-4c2a-4fd8-b990-7a11d594dc0e,13a6c635-2f20-4e22-9e76-c3761d1111be,8605674,1
1,e9b6a2bb-58d4-4e62-8fda-239da00e42a1,1995-06-17 04:27:17,4c1e438a-2b8c-4f84-a22a-869eea346661,84f27bd9-0d26-4f84-b585-cca0e4bbd1d5,8603017,1
2,91a31be2-e71f-436a-917e-1ac65cc9ef2e,2002-04-10 19:26:53,51c3fb8f-c26f-4808-a69a-3f728256cf7a,8c333f9b-af38-4a0c-8bdc-428ffa061312,8763489,1
3,b69901f9-c70b-4133-be43-cc6116f1c8fd,2015-07-31 11:41:37,45e9a922-e602-4e8f-be77-d6486779ae0f,5f9dfbb7-8bca-494a-802e-f11f12c9906a,8607307,1
4,0ccdd3e7-a1cb-43fd-814d-a501278c9f9c,2001-04-06 20:20:43,250343f4-b757-4c3f-aaea-d7d3ddb14744,34d40a00-18f9-4fa5-9973-ffd5d8663c6f,8372995,1
...,...,...,...,...,...,...
6995,b15fcb43-75a5-4a7a-904e-1707686d57a1,1991-01-12 11:28:16,e3d1e9ab-01fb-4ade-ae58-8449f8fbe742,6d1012bc-d57d-4095-87f9-8ef9767987d5,8561587,0
6996,4b5d39cf-2b39-4b84-8052-d94b14193f69,2004-04-21 20:41:31,807a90a6-3103-4cef-944a-304a663b58e1,30edad71-d3f3-479f-87ad-f093880ccf80,8667378,1
6997,50349b56-4dbe-4da4-8145-4abd1f485b94,1977-01-02 22:35:35,81bf0b99-99aa-4211-92be-2315810ee347,cc410d52-c17b-4b93-b07e-e217a75738c6,8487421,1
6998,11c4e42c-048b-454f-9dba-8943234fb752,1986-09-17 09:24:18,8e2cb852-2d27-4a31-8c63-349f9061e641,0c504bb9-49a6-4676-978c-95364a3f06e5,8752744,0


In [9]:
recomendations_df.isnull().sum()

reco_id           0
time_reco         0
id_sender         0
id_reciever       0
id_product        0
product_bought    0
dtype: int64

In [10]:
recomendations_by_sender_bought = recomendations_df.groupby(['id_sender']).sum()

In [11]:
recomendations_by_sender_all = recomendations_df.groupby(['id_sender']).count()

In [12]:
recomendations_by_sender_all = recomendations_df.groupby(['id_sender']).count()

In [13]:
recomendations_by_product_bought = recomendations_df.groupby(['id_product']).sum()
recomendations_by_product_all = recomendations_df.groupby(['id_product']).count()


In [16]:
parameters_to_include =['brand',  'productnature', 'family', 'Review-AverageRating', 'DisplayPrice-Amount', 'ListPrice-Amount']

In [18]:
def add_column(df1,df2, common_value_df1, common_value_df2, parameter_to_add,  name_column):
    ''' Function to add a column to df2 according to a value
            INPUT:  - dataframe to be completed
                    - dataframe to extract info from
                    - common column name in df1
                    - common column name in df2
                    - parameter to add (column from df2)
                    - name of new column
            OUTPUT new df with a column added'''

    new_df=df1.copy()
    new_df.sort_values(common_value_df1, inplace = True)
    index_df1= new_df[common_value_df1].values
    index_df1= list(index_df1)
    index_df2= list(df2[common_value_df2].values)
    list_values= []
    for i in index_df1:
        if i in index_df2:
            new_value = df2[df2[common_value_df2]==i][parameter_to_add].values[0]
            list_values.append(new_value)
        else:
            list_values.append(0)
    new_df[name_column] = list_values
    return new_df
  

In [20]:
#Add columns from product dataframe to recomendations
parameters_to_include =['brand',  'productnature', 'family', 'Review-AverageRating', 'DisplayPrice-Amount', 'ListPrice-Amount']
full_df = recomendations_df.copy()
for i in parameters_to_include:
    full_df = add_column(full_df, products_df, 'id_product', 'modeild',  i, i)


In [21]:
full_df.columns

Index(['reco_id', 'time_reco', 'id_sender', 'id_reciever', 'id_product',
       'product_bought', 'brand', 'productnature', 'family',
       'Review-AverageRating', 'DisplayPrice-Amount', 'ListPrice-Amount'],
      dtype='object')

In [22]:
users_df.columns

Index(['person_id', 'complete_name', 'age', 'address'], dtype='object')

In [23]:
#Add columns from users who give advice to recomendations dataframe
columns_to_add2= ['age', 'address']
for i in columns_to_add2: 
    name_column= i + '_sender'
    full_df = add_column(full_df, users_df, 'id_sender', 'person_id',  i, name_column)


In [24]:
#Add columns from users who ask for advice to recomendations dataframe
columns_to_add2= ['age', 'address']
for i in columns_to_add2: 
    name_column= i + '_reciever'
    full_df = add_column(full_df, users_df, 'id_reciever', 'person_id',  i, name_column)

In [25]:
recomend_user_prod_df=full_df.copy()

In [26]:
recomend_user_prod_df.to_csv(r'C:\Users\march\000 Treball Ju`yter 000\Hackathons\Hackathon_Decathlon_ITAcademy\full_all_df_0.csv')

# Generaring a Ratio for recommenders

In [43]:
user_recomendation = pd.DataFrame(recomend_user_prod_df.groupby('id_sender').count()['reco_id'])
user_recomendation.reset_index(inplace=True)
user_recomendation.columns=['id_sender','Num_recomendations']
user_recomendation


,id_sender,Num_recomendations
0,01ba0eca-6156-47e2-b168-1e3f65815205,32
1,02676c43-e1d6-4857-951b-bd4695357d03,22
2,02f0f461-6ab4-44fc-ae62-f7e5436b62d8,16
3,045e61c1-58da-47af-8c78-433a49d5f7ed,33
4,04cabeab-589e-4d94-9975-21dedb5b2cdc,23
...,...,...
295,fcef0602-cff9-446a-ab89-7048d7a161c0,29
296,fe41517e-bae5-4806-9496-1483593e3f6f,23
297,fe800ee8-68cd-4242-b799-6411a3da6865,19
298,fe916ca1-7bd2-4dc0-b91b-41115e413de9,26


In [44]:


user_recomendation_bought = pd.DataFrame(recomend_user_prod_df.groupby('id_sender').sum()['product_bought'])
user_recomendation_bought.reset_index(inplace=True)

user_recomendation_bought.columns=['id_sender','bougth']
user_recomendation_bought

,id_sender,bougth
0,01ba0eca-6156-47e2-b168-1e3f65815205,20
1,02676c43-e1d6-4857-951b-bd4695357d03,12
2,02f0f461-6ab4-44fc-ae62-f7e5436b62d8,11
3,045e61c1-58da-47af-8c78-433a49d5f7ed,21
4,04cabeab-589e-4d94-9975-21dedb5b2cdc,18
...,...,...
295,fcef0602-cff9-446a-ab89-7048d7a161c0,15
296,fe41517e-bae5-4806-9496-1483593e3f6f,14
297,fe800ee8-68cd-4242-b799-6411a3da6865,9
298,fe916ca1-7bd2-4dc0-b91b-41115e413de9,21


In [45]:
user_recomendation_df = add_column(user_recomendation, user_recomendation_bought, 'id_sender', 'id_sender',  'bougth', 'product_bought')

In [46]:
user_recomendation_df.columns

Index(['id_sender', 'Num_recomendations', 'product_bought'], dtype='object')

In [47]:
max_recomendations=user_recomendation_df['Num_recomendations'] .values.max()
max_recomendations

37

In [48]:
user_recomendation_df['ratio_recommender'] = (user_recomendation_df['Num_recomendations']/max_recomendations
)*user_recomendation_df['product_bought'] /user_recomendation_df['Num_recomendations'] 

In [49]:
user_recomendation_df

,id_sender,Num_recomendations,product_bought,ratio_recommender
0,01ba0eca-6156-47e2-b168-1e3f65815205,32,20,0.540541
1,02676c43-e1d6-4857-951b-bd4695357d03,22,12,0.324324
2,02f0f461-6ab4-44fc-ae62-f7e5436b62d8,16,11,0.297297
3,045e61c1-58da-47af-8c78-433a49d5f7ed,33,21,0.567568
4,04cabeab-589e-4d94-9975-21dedb5b2cdc,23,18,0.486486
...,...,...,...,...
295,fcef0602-cff9-446a-ab89-7048d7a161c0,29,15,0.405405
296,fe41517e-bae5-4806-9496-1483593e3f6f,23,14,0.378378
297,fe800ee8-68cd-4242-b799-6411a3da6865,19,9,0.243243
298,fe916ca1-7bd2-4dc0-b91b-41115e413de9,26,21,0.567568


# Generaring a Ratio for possible customers

In [37]:
user_recomendation_received = pd.DataFrame(recomend_user_prod_df.groupby('id_reciever').count()['reco_id'])
user_recomendation_received.reset_index(inplace=True)
user_recomendation_received.columns=['id_reciever','Num_recomendations']
user_recomendation_received


,id_reciever,Num_recomendations
0,01ba0eca-6156-47e2-b168-1e3f65815205,22
1,02676c43-e1d6-4857-951b-bd4695357d03,25
2,02f0f461-6ab4-44fc-ae62-f7e5436b62d8,33
3,045e61c1-58da-47af-8c78-433a49d5f7ed,36
4,04cabeab-589e-4d94-9975-21dedb5b2cdc,25
...,...,...
295,fcef0602-cff9-446a-ab89-7048d7a161c0,20
296,fe41517e-bae5-4806-9496-1483593e3f6f,18
297,fe800ee8-68cd-4242-b799-6411a3da6865,26
298,fe916ca1-7bd2-4dc0-b91b-41115e413de9,25


In [38]:

user_recomendation_receiver_bought = pd.DataFrame(recomend_user_prod_df.groupby('id_reciever').sum()['product_bought'])
user_recomendation_receiver_bought.reset_index(inplace=True)

user_recomendation_receiver_bought.columns=['id_reciever','bougth']
user_recomendation_receiver_bought

,id_reciever,bougth
0,01ba0eca-6156-47e2-b168-1e3f65815205,16
1,02676c43-e1d6-4857-951b-bd4695357d03,14
2,02f0f461-6ab4-44fc-ae62-f7e5436b62d8,25
3,045e61c1-58da-47af-8c78-433a49d5f7ed,22
4,04cabeab-589e-4d94-9975-21dedb5b2cdc,18
...,...,...
295,fcef0602-cff9-446a-ab89-7048d7a161c0,10
296,fe41517e-bae5-4806-9496-1483593e3f6f,10
297,fe800ee8-68cd-4242-b799-6411a3da6865,14
298,fe916ca1-7bd2-4dc0-b91b-41115e413de9,13


In [39]:
user_recomended_df = add_column(user_recomendation_received, user_recomendation_receiver_bought, 'id_reciever', 'id_reciever',  'bougth', 'product_bought')

In [40]:
user_recomended_df

,id_reciever,Num_recomendations,product_bought
0,01ba0eca-6156-47e2-b168-1e3f65815205,22,16
1,02676c43-e1d6-4857-951b-bd4695357d03,25,14
2,02f0f461-6ab4-44fc-ae62-f7e5436b62d8,33,25
3,045e61c1-58da-47af-8c78-433a49d5f7ed,36,22
4,04cabeab-589e-4d94-9975-21dedb5b2cdc,25,18
...,...,...,...
295,fcef0602-cff9-446a-ab89-7048d7a161c0,20,10
296,fe41517e-bae5-4806-9496-1483593e3f6f,18,10
297,fe800ee8-68cd-4242-b799-6411a3da6865,26,14
298,fe916ca1-7bd2-4dc0-b91b-41115e413de9,25,13


In [41]:
user_recomended_df['ratio_costumer'] = user_recomended_df['product_bought'] /user_recomended_df['Num_recomendations'] 

In [42]:
user_recomended_df

,id_reciever,Num_recomendations,product_bought,ratio_costumer
0,01ba0eca-6156-47e2-b168-1e3f65815205,22,16,0.727273
1,02676c43-e1d6-4857-951b-bd4695357d03,25,14,0.560000
2,02f0f461-6ab4-44fc-ae62-f7e5436b62d8,33,25,0.757576
3,045e61c1-58da-47af-8c78-433a49d5f7ed,36,22,0.611111
4,04cabeab-589e-4d94-9975-21dedb5b2cdc,25,18,0.720000
...,...,...,...,...
295,fcef0602-cff9-446a-ab89-7048d7a161c0,20,10,0.500000
296,fe41517e-bae5-4806-9496-1483593e3f6f,18,10,0.555556
297,fe800ee8-68cd-4242-b799-6411a3da6865,26,14,0.538462
298,fe916ca1-7bd2-4dc0-b91b-41115e413de9,25,13,0.520000


# Data frame to use the model

In [76]:
df_to_model=recomend_user_prod_df.copy()

In [77]:
user_recomendation_df


,id_sender,Num_recomendations,product_bought,ratio_recommender
0,01ba0eca-6156-47e2-b168-1e3f65815205,32,20,0.540541
1,02676c43-e1d6-4857-951b-bd4695357d03,22,12,0.324324
2,02f0f461-6ab4-44fc-ae62-f7e5436b62d8,16,11,0.297297
3,045e61c1-58da-47af-8c78-433a49d5f7ed,33,21,0.567568
4,04cabeab-589e-4d94-9975-21dedb5b2cdc,23,18,0.486486
...,...,...,...,...
295,fcef0602-cff9-446a-ab89-7048d7a161c0,29,15,0.405405
296,fe41517e-bae5-4806-9496-1483593e3f6f,23,14,0.378378
297,fe800ee8-68cd-4242-b799-6411a3da6865,19,9,0.243243
298,fe916ca1-7bd2-4dc0-b91b-41115e413de9,26,21,0.567568


In [78]:
df_to_model['diff_age']=np.abs(df_to_model['age_reciever']-df_to_model['age_sender'])

In [79]:
df_to_model

,reco_id,time_reco,id_sender,id_reciever,id_product,product_bought,brand,productnature,family,Review-AverageRating,DisplayPrice-Amount,ListPrice-Amount,age_sender,address_sender,age_reciever,address_reciever,diff_age
4400,b038a638-3770-4a4b-a408-3a6c8cab2888,1976-12-06 10:07:41,a5380204-bfb8-413d-9013-f547a37304a1,01ba0eca-6156-47e2-b168-1e3f65815205,8487842,1,ARTENGO,ZAPATILLAS,CALZADO HOMBRE MULTICANCHA EXPERTO,4.2,69.99,69.99,16,"2645 William Parks\nErinburgh, AZ 15347",27,Unit 9479 Box 4188\nDPO AE 33020,11
1304,14421056-07d2-44df-b048-3460caa1bdc4,1992-01-03 20:44:32,df8492a7-a27e-47b6-b705-1d468e43c839,01ba0eca-6156-47e2-b168-1e3f65815205,1091517,0,YASAKA,Revestimiento,MADERAS/REVESTIMIENTOS/ACCESORIOS TDM,4.0,24.99,34.99,17,"31869 James Branch\nDavidchester, ID 91031",27,Unit 9479 Box 4188\nDPO AE 33020,10
21,c19b90bf-bd3c-421c-9e95-7dc5bebfe890,2000-07-29 21:33:34,b66d2104-20fd-47a6-b801-564963fb077f,01ba0eca-6156-47e2-b168-1e3f65815205,8598014,1,RSTA,SERVICIOS,PALETA GOMA MACIZA,3.8,19.99,19.99,18,"8315 Juan Row Suite 005\nAllenland, MD 27589",27,Unit 9479 Box 4188\nDPO AE 33020,9
3978,80e84f50-fa2f-429f-9a16-8f2b8f072f36,1998-08-29 08:02:56,993a6d5c-70ec-48e2-aa84-065fd173fd39,01ba0eca-6156-47e2-b168-1e3f65815205,8546095,1,PERFLY,CAMISETAS MANGA CORTA,Ropa badminton mujer,4.1,11.99,11.99,21,"693 Parker Street\nChristinemouth, AK 63993",27,Unit 9479 Box 4188\nDPO AE 33020,6
3719,a1164ea5-56a7-4e0f-b580-2f05d64d950d,1993-11-23 08:15:36,c8e9b7f0-08b2-4fb8-b29a-abed728ce060,01ba0eca-6156-47e2-b168-1e3f65815205,8621580,0,ASICS,ZAPATILLAS,CALZADO MUJER MULTICANCHA INIC/PERFECC,4.6,39.99,54.99,28,"59982 Jessica Pass\nWayneborough, ND 18719",27,Unit 9479 Box 4188\nDPO AE 33020,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3811,3bdb3767-f8b4-40ab-a884-792123cb7587,2016-04-05 05:37:12,6feb0d18-ad8a-49c4-bfdc-8780db59b41c,fed0c95b-74b6-4087-891b-dee1483f5f5d,8667487,0,ASICS,ZAPATILLAS,ZAPATILLA SQUASH,0.0,119.99,119.99,28,"509 Hickman Forge Suite 632\nAustinbury, MO 39387",18,"7556 Tiffany Highway\nRussellville, NH 22165",10
3032,86da8008-86ce-4663-bf17-a25fe982696a,1986-10-18 04:42:42,e266e1a6-976f-4a2e-ac30-17f2b0ec8fe4,fed0c95b-74b6-4087-891b-dee1483f5f5d,8588186,1,TIBHAR,Revestimiento,MADERAS/REVESTIMIENTOS/ACCESORIOS TDM,5.0,42.99,42.99,33,"6559 Craig Mountain Apt. 053\nGeorgefort, AR 10661",18,"7556 Tiffany Highway\nRussellville, NH 22165",15
6357,d189988e-e3d1-4bff-853a-e575bf0205c3,2002-05-02 03:50:01,11b18d52-42bd-4cfb-8cc4-05796ffdff49,fed0c95b-74b6-4087-891b-dee1483f5f5d,8604847,0,ARTENGO,Pantalón corto,Textil tenis mujer,4.6,14.99,14.99,31,"6298 Gardner Plaza Suite 315\nWest Charles, AR 91225",18,"7556 Tiffany Highway\nRussellville, NH 22165",13
4618,c01bbeeb-90b5-4c98-8a4b-ad61d644a419,1995-01-20 00:21:21,487b9274-fffd-4896-bf1d-1fb9b3a8d46f,fed0c95b-74b6-4087-891b-dee1483f5f5d,8589448,1,ARTENGO,ZAPATILLAS,CALZADO TENIS JÚNIOR,4.6,24.99,24.99,37,"36593 Ruiz Radial\nNelsonport, OK 98096",18,"7556 Tiffany Highway\nRussellville, NH 22165",19


In [80]:
#include ratio_recommender column
df_to_model = add_column(df_to_model, user_recomendation_df, 'id_sender', 'id_sender',  'ratio_recommender', 'ratio_recommender')

In [81]:
#include ratio_recommender column
df_to_model = add_column(df_to_model, user_recomendation_df, 'id_sender', 'id_sender',  'ratio_recommender', 'ratio_recommender')

In [82]:
#include ratio_costumer column
df_to_model = add_column(df_to_model, user_recomended_df, 'id_reciever', 'id_reciever',  'ratio_costumer', 'ratio_costumer')

In [83]:
df_to_model.columns

Index(['reco_id', 'time_reco', 'id_sender', 'id_reciever', 'id_product',
       'product_bought', 'brand', 'productnature', 'family',
       'Review-AverageRating', 'DisplayPrice-Amount', 'ListPrice-Amount',
       'age_sender', 'address_sender', 'age_reciever', 'address_reciever',
       'diff_age', 'ratio_recommender', 'ratio_costumer'],
      dtype='object')

In [84]:
for i in ['reco_id', 'time_reco', 'id_sender', 'id_reciever', 'id_product',
       'product_bought', 'brand', 'productnature', 'family',
       'Review-AverageRating', 'DisplayPrice-Amount', 'ListPrice-Amount',
       'age_sender', 'address_sender', 'age_reciever', 'address_reciever']:
    print (i + f' :{df_to_model[i].nunique()}')

reco_id :7000
time_reco :7000
id_sender :300
id_reciever :300
id_product :692
product_bought :2
brand :41
productnature :75
family :82
Review-AverageRating :26
DisplayPrice-Amount :96
ListPrice-Amount :96
age_sender :30
address_sender :300
age_reciever :30
address_reciever :300


In [85]:
df_to_model.columns

Index(['reco_id', 'time_reco', 'id_sender', 'id_reciever', 'id_product',
       'product_bought', 'brand', 'productnature', 'family',
       'Review-AverageRating', 'DisplayPrice-Amount', 'ListPrice-Amount',
       'age_sender', 'address_sender', 'age_reciever', 'address_reciever',
       'diff_age', 'ratio_recommender', 'ratio_costumer'],
      dtype='object')

In [89]:
def drop_columns(df):
    df.drop(['reco_id', 
    'time_reco', 
    'id_sender', 
    'id_reciever',
     'id_product',
      'brand', 
      'productnature', 
      'family',
        'address_sender', 
        'age_reciever', 
        'address_reciever'
        ],
        axis=1, inplace=True)
    return df

In [90]:
df_to_model=drop_columns(df_to_model)

In [91]:
df_to_model

,product_bought,Review-AverageRating,DisplayPrice-Amount,ListPrice-Amount,age_sender,diff_age,ratio_recommender,ratio_costumer
4400,1,4.2,69.99,69.99,16,11,0.459459,0.727273
617,0,4.4,2.99,2.99,29,2,0.513514,0.727273
5199,1,5.0,15.99,15.99,36,9,0.351351,0.727273
6399,1,4.1,21.99,27.99,35,8,0.459459,0.727273
3978,1,4.1,11.99,11.99,21,6,0.378378,0.727273
...,...,...,...,...,...,...,...,...
3811,0,0.0,119.99,119.99,28,10,0.243243,0.555556
2753,1,4.4,3.99,3.99,28,10,0.324324,0.555556
552,0,0.0,69.99,69.99,24,6,0.405405,0.555556
172,1,4.5,11.99,15.99,16,2,0.513514,0.555556


In [93]:
# save to share the info
df_to_model.to_csv(r'C:\Users\march\000 Treball Ju`yter 000\Hackathons\Hackathon_Decathlon_ITAcademy\df_to_model.csv')

In [ ]:
def grafic_pca3 (df_norm, labels):
    '''Function to visualize clustering in a 3D graph
            INPUT: a dataframe with normalized values (only numerical values)
                   a list with the labels of clusters 
            OUTPUT: a 3D graph 
    '''
    pca = PCA(n_components=3).fit_transform(df_norm)
    fig = plt.figure(figsize=(12, 7), dpi=80, facecolor='w', edgecolor='k')
    ax = plt.axes(projection="3d")
    ax.scatter3D(pca.T[0],pca.T[1],pca.T[2],c=labels,cmap='Spectral')

    xLabel = ax.set_xlabel('X')
    yLabel = ax.set_ylabel('Y')
    zLabel = ax.set_zlabel('Z')

In [108]:
X=df_to_model.copy()
X.drop('product_bought',axis=1,inplace=True)
numerical_cols=list(X.columns)

In [110]:
robust_scaler = preprocessing.RobustScaler()
X_train_robust = robust_scaler.fit_transform(X)
X_train_robust = pd.DataFrame(X_train_robust, columns =numerical_cols)
X_train_robust

,Review-AverageRating,DisplayPrice-Amount,ListPrice-Amount,age_sender,diff_age,ratio_recommender,ratio_costumer
0,-0.500000,1.093023,1.046512,-0.928571,0.3,0.4,0.770343
1,-0.166667,-0.465116,-0.511628,0.000000,-0.6,0.8,0.770343
2,0.833333,-0.162791,-0.209302,0.500000,0.1,-0.4,0.770343
3,-0.666667,-0.023256,0.069767,0.428571,0.0,0.4,0.770343
4,-0.666667,-0.255814,-0.302326,-0.571429,-0.2,-0.2,0.770343
...,...,...,...,...,...,...,...
6995,-7.500000,2.255814,2.209302,-0.071429,0.2,-1.2,-0.436319
6996,-0.166667,-0.441860,-0.488372,-0.071429,0.2,-0.6,-0.436319
6997,-7.500000,1.093023,1.046512,-0.357143,-0.2,0.0,-0.436319
6998,0.000000,-0.255814,-0.209302,-0.928571,-0.6,0.8,-0.436319


In [113]:
standard_scaler = preprocessing.StandardScaler()
X_train_standard = standard_scaler.fit_transform(X)
X_train_standard = pd.DataFrame(X_train_standard, columns =numerical_cols)

In [119]:
min_max_scaler = preprocessing.MinMaxScaler()
X_train_min_max = min_max_scaler.fit_transform(X)
X_train_min_max = pd.DataFrame(X_train_min_max, columns =numerical_cols)

In [120]:
X_train_min_max ['product_bought'] = df_to_model['product_bought']
X_train_robust ['product_bought'] = df_to_model['product_bought']
X_train_standard ['product_bought'] = df_to_model['product_bought']

# Model screening with Pycaret

In [100]:
# To get modular programation, next funtions are created to perform screening of Models with Pycaret
# to test different initial dataframes and session_id. 

def setup_pycaret(df, session_id):
    ''' funtion to make the setup of Pycaret
    Input: data Frame and session id
    Output: none 
    '''
    setup(df,
          target='product_bought',
          session_id = session_id,
          silent = True
    )


In [101]:

def screen_models_pycaret(df,   name_df, session_id=42):
    ''' funtion to do screening with Pycaret
    Input: data Frame, session_id (by default 42) and name of the initial dataframe
    Output: dicctionary which contains the followin info KEY = CONTENT:
        'best_metod' = the best method of the screening with hyperparameters 
        'results_df' = a pandas dataframe with the results of the screening with all methodologies
    '''
    setup_pycaret(df, session_id)
    best_model=compare_models()
    df_results= pycaret.classification.pull() #extract results as data frame
    df_results.insert(loc = 0, column ='Initial df', value= name_df) #introduce name of initial df
    df_results.reset_index(drop=True, inplace=True) #remove old index
    results_dictionary ={'best_method' : best_model, 'results_df':  df_results }
    return results_dictionary

In [125]:
dict_unmodified_df_1 = screen_models_pycaret(df_to_model, 'original dataframe',1)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lda,Linear Discriminant Analysis,0.7057,0.7226,0.9135,0.7000,0.7926,0.3152,0.3507,0.0280
lr,Logistic Regression,0.7046,0.7224,0.9128,0.6993,0.7919,0.3127,0.3481,0.0730
ridge,Ridge Classifier,0.7036,0.0000,0.9168,0.6973,0.7920,0.3081,0.3458,0.0260
gbc,Gradient Boosting Classifier,0.6985,0.7096,0.8879,0.7017,0.7838,0.3076,0.3324,0.3980
ada,Ada Boost Classifier,0.6965,0.7090,0.8866,0.7002,0.7824,0.3029,0.3272,0.2210
qda,Quadratic Discriminant Analysis,0.6957,0.7140,0.9264,0.6877,0.7894,0.2819,0.3269,0.0310
nb,Naive Bayes,0.6930,0.7139,0.9277,0.6851,0.7882,0.2739,0.3201,0.0330
rf,Random Forest Classifier,0.6855,0.6966,0.8349,0.7072,0.7657,0.2971,0.3071,0.8160
lightgbm,Light Gradient Boosting Machine,0.6795,0.6923,0.8452,0.6981,0.7645,0.2769,0.2900,0.1140
et,Extra Trees Classifier,0.6785,0.6894,0.8339,0.7010,0.7616,0.2793,0.2896,0.6020


In [131]:
dict_standard_df_1 = screen_models_pycaret(X_train_standard , 'Standard scaled',1)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.6134,0.4998,1.0000,0.6134,0.7604,0.0000,0.0000,1.9260
ridge,Ridge Classifier,0.6134,0.0000,1.0000,0.6134,0.7604,0.0000,0.0000,0.0230
lda,Linear Discriminant Analysis,0.6134,0.4983,1.0000,0.6134,0.7604,0.0000,0.0000,0.0250
ada,Ada Boost Classifier,0.6091,0.4941,0.9864,0.6126,0.7558,-0.0037,-0.0093,0.2340
svm,SVM - Linear Kernel,0.6052,0.0000,0.9767,0.6115,0.7518,-0.0089,-0.0231,0.0380
gbc,Gradient Boosting Classifier,0.5971,0.5038,0.9461,0.6107,0.7423,-0.0126,-0.0240,0.4140
nb,Naive Bayes,0.5926,0.4978,0.9159,0.6121,0.7328,-0.0047,-0.0060,0.0230
qda,Quadratic Discriminant Analysis,0.5807,0.5091,0.8394,0.6161,0.7087,0.0114,0.0144,0.0300
lightgbm,Light Gradient Boosting Machine,0.5777,0.5168,0.8293,0.6156,0.7065,0.0087,0.0101,1.8160
rf,Random Forest Classifier,0.5701,0.5150,0.7970,0.6154,0.6944,0.0080,0.0092,0.8220


In [127]:
dict_unmodified_df_1 = screen_models_pycaret(df_to_model, 'original dataframe',1)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lda,Linear Discriminant Analysis,0.7057,0.7226,0.9135,0.7000,0.7926,0.3152,0.3507,0.0260
lr,Logistic Regression,0.7046,0.7224,0.9128,0.6993,0.7919,0.3127,0.3481,0.0770
ridge,Ridge Classifier,0.7036,0.0000,0.9168,0.6973,0.7920,0.3081,0.3458,0.0200
gbc,Gradient Boosting Classifier,0.6985,0.7096,0.8879,0.7017,0.7838,0.3076,0.3324,0.4230
ada,Ada Boost Classifier,0.6965,0.7090,0.8866,0.7002,0.7824,0.3029,0.3272,0.1920
qda,Quadratic Discriminant Analysis,0.6957,0.7140,0.9264,0.6877,0.7894,0.2819,0.3269,0.0250
nb,Naive Bayes,0.6930,0.7139,0.9277,0.6851,0.7882,0.2739,0.3201,0.0220
rf,Random Forest Classifier,0.6855,0.6966,0.8349,0.7072,0.7657,0.2971,0.3071,0.6360
lightgbm,Light Gradient Boosting Machine,0.6795,0.6923,0.8452,0.6981,0.7645,0.2769,0.2900,0.0960
et,Extra Trees Classifier,0.6785,0.6894,0.8339,0.7010,0.7616,0.2793,0.2896,0.6180


In [123]:
dict_min_max_df_1 = screen_models_pycaret(X_train_min_max, 'Min_max scaled',1)


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.6134,0.4973,1.0000,0.6134,0.7604,0.0000,0.0000,0.0410
svm,SVM - Linear Kernel,0.6134,0.0000,1.0000,0.6134,0.7604,0.0000,0.0000,0.0490
ridge,Ridge Classifier,0.6134,0.0000,1.0000,0.6134,0.7604,0.0000,0.0000,0.0230
lda,Linear Discriminant Analysis,0.6134,0.4983,1.0000,0.6134,0.7604,0.0000,0.0000,0.0240
ada,Ada Boost Classifier,0.6091,0.4941,0.9864,0.6126,0.7558,-0.0037,-0.0093,0.2690
gbc,Gradient Boosting Classifier,0.5969,0.5039,0.9458,0.6107,0.7421,-0.0130,-0.0247,0.3800
nb,Naive Bayes,0.5926,0.4979,0.9159,0.6121,0.7328,-0.0047,-0.0060,0.0230
qda,Quadratic Discriminant Analysis,0.5807,0.5091,0.8394,0.6161,0.7087,0.0114,0.0144,0.0230
lightgbm,Light Gradient Boosting Machine,0.5777,0.5168,0.8293,0.6156,0.7065,0.0087,0.0101,0.0960
rf,Random Forest Classifier,0.5711,0.5165,0.7960,0.6164,0.6947,0.0115,0.0132,0.8620


In [122]:
dict_robust_df_1 = screen_models_pycaret(X_train_robust, 'Robust scaled',1)


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.6134,0.4998,1.0000,0.6134,0.7604,0.0000,0.0000,0.0300
ridge,Ridge Classifier,0.6134,0.0000,1.0000,0.6134,0.7604,0.0000,0.0000,0.0240
lda,Linear Discriminant Analysis,0.6134,0.4983,1.0000,0.6134,0.7604,0.0000,0.0000,0.0250
ada,Ada Boost Classifier,0.6091,0.4941,0.9864,0.6126,0.7558,-0.0037,-0.0093,0.2410
gbc,Gradient Boosting Classifier,0.5969,0.5038,0.9458,0.6107,0.7421,-0.0130,-0.0247,0.4310
nb,Naive Bayes,0.5926,0.4979,0.9159,0.6121,0.7328,-0.0047,-0.0060,0.0260
qda,Quadratic Discriminant Analysis,0.5807,0.5091,0.8394,0.6161,0.7087,0.0114,0.0144,0.0220
lightgbm,Light Gradient Boosting Machine,0.5791,0.5149,0.8223,0.6178,0.7055,0.0173,0.0201,0.1600
svm,SVM - Linear Kernel,0.5783,0.0000,0.8559,0.6114,0.7124,-0.0067,-0.0055,0.0410
rf,Random Forest Classifier,0.5707,0.5164,0.7970,0.6159,0.6947,0.0097,0.0111,0.9010


Let's compile results in one dataframe:

In [130]:
dict_unmodified_df_1['results_df'].iloc[:2,:]

,Initial df,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
0,original dataframe,Linear Discriminant Analysis,0.7057,0.7226,0.9135,0.7000,0.7926,0.3152,0.3507,0.026
1,original dataframe,Logistic Regression,0.7046,0.7224,0.9128,0.6993,0.7919,0.3127,0.3481,0.077


In [ ]:
all_results_screening_pycaret_initial_state1=pd.DataFrame()
for dictionary in [dict_unmodified_df_1, 
                    dict_robust_df_1,
                    dict_standard_df_1 ,
                     
                    ]:
    join=[all_results_screening_pycaret_initial_state1, dictionary['results_df'].iloc[:2,:]]
    all_results_screening_pycaret_initial_state1 = pd.concat(join)
all_results_screening_pycaret_initial_state1.reset_index(drop=True, inplace=True)
all_results_screening_pycaret_initial_state1

NameError: name 'dict_standard_df_1' is not defined

## test_X prediction

As seen in the previous section, the best method is the Quadratic Discriminant Analysis. We proceed to fit with the whole training dataset standarized to get the best performance:

In [48]:
y_training=X_train_standard.diagnosis
x_training=X_train_standard.copy()
x_training.drop('diagnosis',axis=1, inplace=True)

In [49]:
best_QDA.fit(x_training, y_training)

QuadraticDiscriminantAnalysis(priors=None, reg_param=0.05,
                              store_covariance=False, tol=0.0001)

Charge test_X dataset to be predicted. We check its statistically ressemblance to the training data set.:

In [133]:
test_x=pd.read_csv(r'C:\Users\march\000 Treball Ju`yter 000\Hackathons\Hackathon_Decathlon_ITAcademy\Docs Inicials\prods_IT.csv', index_col=None)
test_x

,modeild,weblabel,brand,catchline,productnature,family,Review-AverageRating,DisplayPrice-Amount,ListPrice-Amount
0,8071204,REGULADOR DE RED DE TENIS,ARTENGO,Indispensable para una buena altura de red.,REDES,EQUIPAMIENTO COACH/CLUB,5.0,9.99,9.99
1,8071566,PELOTA DE TENIS POLIVALENTE TB920 *4 AMARILLO,ARTENGO,"¡Durabilidad y control!Pelota oficial del torneo de la ATP 250 Moselle Open, la TB920 se recomienda por su calidad de rebote y por su control.Nuevo empaquetado y nuevo logotipo pero pelotas idénticas.",Pelota de tenis,PELOTAS TENIS,4.5,5.99,5.99
2,8071569,PELOTA DE TENIS TB530 *4 AMARILLO,ARTENGO,¡Pelota de competición a bajo precio!Nuevo empaquetado y nuevo logotipo pero pelotas idénticas.,Pelota de tenis,PELOTAS TENIS,4.5,4.49,4.49
3,8072596,"CORDAJE DE TENIS BABOLAT MULTIFILAMENTOS XCEL LIMITED 1,30 mm NATURAL",BABOLAT,"Este cordaje de tenis multifilamentos Babolat Xcel Limited 1,3 mm es perfecto para los jugadores que buscan comodidad y sensaciones.",Cordaje raqueta,CORDAJES TENIS,4.7,14.99,14.99
4,8072605,"Cordaje Tenis Monofilamento Big Banger Alu Power Gris 1,25 mm Gris",LUXILON,"Este cordaje de tenis monofilamento Luxilon Alu Power de 1,25 mm es el cordaje más utilizado por los expertos gracias a su potencia y control de alta gama.",Cordaje raqueta,CORDAJES TENIS,4.3,14.99,14.99
...,...,...,...,...,...,...,...,...,...
4861,8030841,"Pelota de Speedball ""TURNBALL SLOW BALL"" espuma amarillo",ARTENGO,"La pelota de espuma ofrece la velocidad ideal para el aprendizaje del speedball.Aconsejado para niños menores de 10 años, se monta muy fácilmente y se adapta a todos los turnball Artengo.",Speedball ball,SPEEDBALL,4.7,3.99,3.99
4862,1093307,REVESTIMIENTO PALA DE PING PONG VARI SPIN,TIBHAR,Una adherencia excepcional para todos.,Revestimiento,MADERAS/REVESTIMIENTOS/ACCESORIOS TDM,4.7,29.99,29.99
4863,1093307,REVESTIMIENTO PALA DE PING PONG VARI SPIN,TIBHAR,Una adherencia excepcional para todos.,Revestimiento,MADERAS/REVESTIMIENTOS/ACCESORIOS TDM,4.7,29.99,29.99
4864,1095016,MADERA PALA DE PING-PONG IV L,TIBHAR,¡Madera de ping-pong Tibhar para todos!\nIdeal para aprender las bases del ping-pong gracias a un excelente control. \nAdecuada para todos los estilos de juego.,Madera,MADERAS/REVESTIMIENTOS/ACCESORIOS TDM,4.0,29.99,29.99


In [134]:
#Add columns from product dataframe to recomendations
parameters_to_include =['brand',  'productnature', 'family', 'Review-AverageRating', 'DisplayPrice-Amount', 'ListPrice-Amount']

for i in parameters_to_include:
    test_x = add_column(test_x, products_df, 'id_product', 'modeild',  i, i)

KeyError: 'id_product'

In [ ]:
r'C:\Users\march\000 Treball Ju`yter 000\Hackathons

In [51]:
test_x.shape

(114, 30)

In [52]:
test_x.describe()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
count,114.000000,114.000000,114.000000,114.000000,114.000000,114.000000,114.000000,114.000000,114.000000,114.000000,...,114.000000,114.000000,114.000000,114.000000,114.000000,114.000000,114.000000,114.000000,114.000000,114.000000
mean,14.080561,19.223246,91.502018,643.151754,0.094411,0.099489,0.080914,0.046263,0.180534,0.062289,...,16.239298,25.095000,107.028509,872.720175,0.127924,0.243361,0.248026,0.110261,0.289082,0.082495
std,3.098263,4.451322,21.375850,302.252445,0.014287,0.052283,0.074235,0.037399,0.028599,0.007128,...,4.586637,6.228466,32.000737,549.816427,0.022095,0.156545,0.194433,0.064967,0.070150,0.017634
min,8.597000,10.380000,54.090000,221.200000,0.064290,0.023440,0.000000,0.000000,0.106000,0.050240,...,8.952000,12.490000,56.650000,240.100000,0.071170,0.027290,0.000000,0.000000,0.156600,0.058650
25%,11.697500,16.040000,75.582500,421.475000,0.083950,0.061023,0.025683,0.018045,0.161725,0.056755,...,13.070000,20.685000,83.787500,523.975000,0.113775,0.117275,0.092397,0.062960,0.246225,0.070930
50%,13.500000,18.760000,87.470000,555.150000,0.092455,0.085920,0.054505,0.032100,0.176700,0.061460,...,14.880000,25.265000,97.505000,675.150000,0.128450,0.210700,0.193950,0.099575,0.277850,0.078155
75%,16.045000,21.780000,105.550000,801.075000,0.102250,0.127825,0.110250,0.065710,0.195175,0.065457,...,19.695000,29.062500,128.800000,1198.750000,0.141175,0.341475,0.358100,0.157750,0.319375,0.091390
max,25.730000,33.810000,174.200000,2010.000000,0.142500,0.283900,0.336800,0.191300,0.274300,0.097440,...,33.130000,47.160000,229.300000,3432.000000,0.209800,0.866300,0.772700,0.275600,0.663800,0.173000


In [53]:
train_diag_Cancer_df.drop('diagnosis',axis=1).describe()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
count,455.000000,455.000000,455.000000,455.000000,455.000000,455.000000,455.000000,455.000000,455.000000,455.000000,...,455.000000,455.000000,455.000000,455.000000,455.000000,455.000000,455.000000,455.000000,455.000000,455.000000
mean,14.139000,19.306286,92.086044,657.829890,0.096849,0.105557,0.090775,0.049585,0.181319,0.062925,...,16.276679,25.823099,107.319516,882.553187,0.133482,0.256997,0.278242,0.115695,0.290325,0.084309
std,3.625935,4.267421,24.998077,363.536865,0.013981,0.052932,0.080993,0.039159,0.027140,0.007045,...,4.897917,6.123728,34.025815,574.721873,0.022902,0.157587,0.211802,0.065949,0.059696,0.018168
min,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.116700,0.049960,...,7.930000,12.020000,50.410000,185.200000,0.081250,0.034320,0.000000,0.000000,0.156500,0.055040
25%,11.705000,16.210000,75.015000,419.250000,0.086740,0.066735,0.030180,0.020720,0.162050,0.057885,...,12.980000,21.350000,84.290000,513.950000,0.118350,0.151650,0.119350,0.065080,0.251550,0.071835
50%,13.300000,18.890000,85.980000,546.400000,0.096860,0.094860,0.063870,0.033840,0.179700,0.061550,...,14.990000,25.440000,97.660000,688.600000,0.132600,0.215600,0.229900,0.099930,0.283300,0.080190
75%,15.740000,21.795000,103.650000,777.250000,0.106050,0.130450,0.132200,0.074490,0.195800,0.066375,...,18.550000,29.990000,124.950000,1047.000000,0.147950,0.333800,0.387200,0.161950,0.317500,0.092070
max,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,0.095750,...,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.577400,0.207500


Statiscally, features in test_x are very similar to  the training set. Then, it can be proceed as the training to get the predictions:
    1st standarize the dataset
    2nd apply the best model fitted with the whole training data set.

In [54]:
X_test_standard = standard_scaler.transform(test_x)
X_test_standard = pd.DataFrame(X_test_standard, columns =numerical_cols)
X_test_standard.describe()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
count,114.000000,114.000000,114.000000,114.000000,114.000000,114.000000,114.000000,114.000000,114.000000,114.000000,...,114.000000,114.000000,114.000000,114.000000,114.000000,114.000000,114.000000,114.000000,114.000000,114.000000
mean,-0.016135,-0.019481,-0.023389,-0.040420,-0.174581,-0.114761,-0.121882,-0.084911,-0.028953,-0.090308,...,-0.007640,-0.119029,-0.008562,-0.017128,-0.242933,-0.086622,-0.142821,-0.082487,-0.020846,-0.099975
std,0.855413,1.044242,0.856041,0.832337,1.022982,0.988828,0.917563,0.956119,1.054944,1.012855,...,0.937477,1.018223,0.941519,0.957718,0.965818,0.994477,0.919004,0.986192,1.176417,0.971683
min,-1.530116,-2.094031,-1.521632,-1.202383,-2.331333,-1.553070,-1.122005,-1.267647,-2.778309,-1.802433,...,-1.497114,-2.179681,-1.490788,-1.119081,-2.723781,-1.459252,-1.315135,-1.756246,-2.242576,-1.413915
25%,-0.674085,-0.766243,-0.660919,-0.650869,-0.923601,-0.842274,-0.804562,-0.806320,-0.722772,-0.876702,...,-0.655423,-0.839971,-0.692354,-0.624603,-0.861434,-0.887607,-0.878411,-0.800515,-0.739555,-0.737246
50%,-0.176424,-0.128154,-0.184859,-0.282758,-0.314611,-0.371388,-0.448307,-0.446999,-0.170386,-0.208157,...,-0.285472,-0.091237,-0.288761,-0.361273,-0.219961,-0.294109,-0.398415,-0.244700,-0.209201,-0.339124
75%,0.526236,0.580312,0.539193,0.394466,0.386749,0.421160,0.240718,0.412252,0.511104,0.359857,...,0.698681,0.529574,0.631994,0.550779,0.336273,0.536661,0.377453,0.638395,0.487178,0.390169
max,3.200211,3.402448,3.288427,3.723580,3.268804,3.373001,3.040943,3.623004,3.429801,4.904323,...,3.444704,3.488135,3.588885,4.440849,3.336005,3.870699,2.337092,2.427356,6.263221,4.887156


In [55]:
train_diag_Cancer_df

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,0,10.710,20.39,69.50,344.9,0.10820,0.12890,0.08448,0.02867,0.1668,...,11.690,25.21,76.51,410.4,0.1335,0.2550,0.2534,0.08600,0.2605,0.08701
1,0,13.640,16.34,87.21,571.8,0.07685,0.06059,0.01857,0.01723,0.1353,...,14.670,23.19,96.08,656.7,0.1089,0.1582,0.1050,0.08586,0.2346,0.08025
2,0,11.710,17.19,74.68,420.3,0.09774,0.06141,0.03809,0.03239,0.1516,...,13.010,21.39,84.42,521.5,0.1323,0.1040,0.1521,0.10990,0.2572,0.07097
3,1,15.300,25.27,102.40,732.4,0.10820,0.16970,0.16830,0.08751,0.1926,...,20.270,36.71,149.30,1269.0,0.1641,0.6110,0.6335,0.20240,0.4027,0.09876
4,1,14.450,20.22,94.49,642.7,0.09872,0.12060,0.11800,0.05980,0.1950,...,18.330,30.12,117.90,1044.0,0.1552,0.4056,0.4967,0.18380,0.4753,0.10130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450,1,13.170,21.81,85.42,531.5,0.09714,0.10470,0.08259,0.05252,0.1746,...,16.230,29.89,105.50,740.7,0.1503,0.3904,0.3728,0.16070,0.3693,0.09618
451,0,15.100,16.39,99.58,674.5,0.11500,0.18070,0.11380,0.08534,0.2001,...,16.110,18.33,105.90,762.6,0.1386,0.2883,0.1960,0.14230,0.2590,0.07779
452,1,14.710,21.59,95.55,656.9,0.11370,0.13650,0.12930,0.08123,0.2027,...,17.870,30.70,115.70,985.5,0.1368,0.4290,0.3587,0.18340,0.3698,0.10940
453,0,9.606,16.84,61.64,280.5,0.08481,0.09228,0.08422,0.02292,0.2036,...,10.750,23.07,71.25,353.6,0.1233,0.3416,0.4341,0.08120,0.2982,0.09825


In [56]:
x_test_pred = best_QDA.predict(X_test_standard)
x_test_pred

array([0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 0, 1, 0], dtype=int64)

In [57]:
predictions_df=pd.DataFrame(x_test_pred,columns=['diagnosis'])
predictions_df

,diagnosis
0,0
1,0
2,0
3,1
4,0
...,...
109,0
110,0
111,0
112,1


In [58]:
predictions_df.to_csv(r'C:\Users\march\000 Treball Ju`yter 000\Hackathons\Access_Project_Hackathon_IT_Decathlon\predictions.csv',index=False)

In [135]:
pip list > requirements.txt #generates file with all packages installed in the environment used

Note: you may need to restart the kernel to use updated packages.
